In [1]:
import cv2

def main():
    template = cv2.imread('WhatsApp Image.jpg', cv2.IMREAD_GRAYSCALE)
    template_height, template_width = template.shape

    orb = cv2.ORB_create()
    keypoints_template, descriptors_template = orb.detectAndCompute(template, None)

    # FLANN parameters
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50) 

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    cap = cv2.VideoCapture(0)  # Change the index to your camera's index (usually 0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        keypoints_frame, descriptors_frame = orb.detectAndCompute(gray_frame, None)

        if descriptors_frame is None:
            continue

        matches = flann.knnMatch(descriptors_template, descriptors_frame, k=2)

        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < 0.7 * n.distance:  # Lowe's ratio test
                    good_matches.append(m)

        if len(good_matches) >= 10:
            matching_result = cv2.drawMatches(template, keypoints_template, gray_frame, keypoints_frame, good_matches, None,
                                             flags=2)
            cv2.imshow('Object Detection', matching_result)
        else:
            cv2.imshow('Object Detection', gray_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()




In [2]:
if __name__ == '__main__':
    main()